In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import pymongo
import re

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
def get_soup(url):
    browser = init_browser()

    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    browser.quit()

    return soup

In [4]:
def scrape():
    
    sbn = {}
    temp = []
    
    sbn_url = "https://www.strawberrynet.com"
    
    urls = [{"eyes_lips":"/en-us/skincare/?groupid=3&sort=popularity"},
            {"cleansers":"/en-us/skincare/cleansers/t/?groupid=4&sort=popularity"},
            {"masks":"/en-us/skincare/masks/t/?groupid=4&sort=popularity"},
            {"moisturisers":"/en-us/skincare/moisturizers-treatments/t/?groupid=4&sort=popularity"},
            {"sun_care":"/en-us/skincare/?groupid=7&sort=popularity"}
           ]
    
    for url in urls:
        for key, value in url.items():
            temp = []
            browser = init_browser()
            browser.visit(sbn_url+value)

            html = browser.html
            soup = BeautifulSoup(html, "html.parser")

            #Get last page of product
            pages = soup.find_all("a",class_="gopage")
            last_page = (pages[-1].get("onclick")).split("'")
            l = int(last_page[1]) + 1
            i=1

            #Loop through all pages to get product links    
            for i in range(l):
                
                if (i!=1):
                    browser.execute_script("goPage("+str(i)+");return false;")
                    time.sleep(1)
                    html = browser.html
                    soup = BeautifulSoup(html, "html.parser")
                    
                links = soup.find_all("a",class_="prod-name")

                for link in links:
                        temp.append(link.get('href'))

            browser.quit()

            sbn[key] = temp
    
    return sbn

In [5]:
def scrape_product_details(url):
        
    print(url)
    browser = init_browser()
    browser.visit(url)
    
    retry_url = []
    
    try:
    
        product_details = {}
        reviews_details = []
        review_num = 0
        avg_stars = 0

        html = browser.html
        soup = BeautifulSoup(html, "html.parser")

        # Get the brand name
        page = soup.find("div",class_="col-sm-6 col-md-7 col-lg-7")
        brand_name = page.find('a').get('title')

        # Get the product name

        product_name = page.find("h1").text

        # Get the price

        price_int = page.find("span",class_="intPrice font-48").text
        price_dec = page.find("span",class_="decPrice font-20").text
        product_price = price_int+price_dec
        total_stars = 0
        # Get the reviews no, header, content, date

        # Check if there are any reviews



        find_review = page.find("span",class_="review-text mr5 ").text

        if find_review:

            # Get the total number of reviews
            review_text = re.findall(r"[\w']+",find_review)
            review_num = int(review_text[0])

            # Click on the Reviews tab
            browser.execute_script("themePromoContentToggle()")

            # Loop through all the pages for reviews

            for i in range(0, (review_num//5)+1):

                if (i!=0):
                    browser.execute_script("goPage("+str(i)+");return false;")
                time.sleep(1)
                html = browser.html
                soup = BeautifulSoup(html, "html.parser")
                page = soup.find("div",class_="col-sm-6 col-md-7 col-lg-7")

                # Collect all the review information per page
                stars, reviews_page = reviews_per_page(page);
                total_stars += stars
                reviews_details.append(reviews_page)

        avg_stars = total_stars/review_num

        product_details = {
                                "product_url": url,
                                "brand_name": brand_name,
                                "product_name": product_name,
                                "product_price": product_price,
                                "review_num": review_num,
                                "avg_stars": avg_stars,
                                "review_details": reviews_details

        }

        
    
    except:
        retry_url.append(url)
        print("Error - Retry: "+ url)
        
    browser.quit()
    
    return product_details, retry_url
        

In [6]:
def reviews_per_page(page):
            # Get the reviews per page
        reviews = page.find_all("article", class_="comments-block")
        
        reviews_page = []
        total_stars = 0
        
        for review in reviews:
            
            each_review = {}
            
            # Get the review date
            review_date = review.find("span", class_="fright ml10 mt-2 grey-color").text
            
            # Get the review header
            review_header = review.find("strong",class_="heading ").text
            
            # Get the review comment
            review_comment = review.find("p", class_="mt10").text
            
            # Get the stars
            img_stars = review.find("span", class_="fleft")
            stars = img_stars.find_all("img")
            stars_num = 0
            

            # Count the number of stars
            for i in range(5):
                star = stars[i].get("src").split('/')
                if star[5]=="star_full.gif":
                    stars_num += 1
            total_stars += stars_num           
            
            each_review = {
                      "review_date": review_date,
                      "review_header": review_header,
                      "review_comment": review_comment,
                      "stars_num": stars_num
            }
            
            reviews_page.append(each_review)   
        
        return total_stars, reviews_page

In [7]:
# Check how many items are in a list
def count_items(a_list):
    category_count = {}

    for key in a_list.keys():
        category_count[key] = len(a_list[key])
    return category_count

# Scrape Strawberrynet Website for links to all products

In [8]:
# Initialize PyMongo to work with MongoDBs
connect = 'mongodb://localhost:27017'
client = pymongo.MongoClient(connect)

In [9]:
# Define database and collection
db = client.sbn_db
c_category = db.category
c_product_details = db.product_details

In [10]:
# Get all the categories and product url links

sbn_product_urls = scrape()
sbn_product_urls

{'eyes_lips': ['/en-us/skincare/elizabeth-arden/eight-hour-lipcare-stick/13488/',
  '/en-us/skincare/shiseido/benefiance-wrinkleresist24-intensive/140188/',
  '/en-us/skincare/clinique/all-about-eye-serum-de-puffing/104889/',
  '/en-us/skincare/l-occitane/shea-butter-lip-balm-stick/42067/',
  '/en-us/skincare/clarins/hydra-essentiel-moisture-replenishing/94452/',
  '/en-us/skincare/clarins/extra-firming-eye-lift-perfecting/167004/',
  '/en-us/skincare/estee-lauder/advanced-night-repair-eye-synchronized/172053/',
  '/en-us/skincare/lancome/bi-facil/13977/',
  '/en-us/skincare/clinique/take-the-day-off-make-up-remover/13236/',
  '/en-us/skincare/clarins/eye-contour-gel/12993/',
  '/en-us/skincare/dermalogica/age-smart-age-reversal-eye-complex/110802/',
  '/en-us/skincare/elizabeth-arden/all-gone-lip-eye-makeup-remover/37773/',
  '/en-us/skincare/gatineau/aquamemory-moisture-replenish-eye/109807/',
  '/en-us/skincare/guinot/eye-lifting/18560/',
  '/en-us/skincare/clinique/all-about-eyes/1

In [12]:
# Remove the extra _id category

sbn_product_urls.pop('_id', None)

In [13]:
# Count scraped items

count = count_items(sbn_product_urls)
count

{'eyes_lips': 833,
 'cleansers': 636,
 'masks': 713,
 'moisturisers': 1612,
 'sun_care': 509}

In [14]:
# Remove duplicates from scraped list

clean_urls = {}

for key in sbn_product_urls.keys():

    clean_urls[key] = sbn_product_urls[key]
    clean_urls[key] = list(dict.fromkeys(clean_urls[key]))
    
clean_urls

{'eyes_lips': ['/en-us/skincare/elizabeth-arden/eight-hour-lipcare-stick/13488/',
  '/en-us/skincare/shiseido/benefiance-wrinkleresist24-intensive/140188/',
  '/en-us/skincare/clinique/all-about-eye-serum-de-puffing/104889/',
  '/en-us/skincare/l-occitane/shea-butter-lip-balm-stick/42067/',
  '/en-us/skincare/clarins/hydra-essentiel-moisture-replenishing/94452/',
  '/en-us/skincare/clarins/extra-firming-eye-lift-perfecting/167004/',
  '/en-us/skincare/estee-lauder/advanced-night-repair-eye-synchronized/172053/',
  '/en-us/skincare/lancome/bi-facil/13977/',
  '/en-us/skincare/clinique/take-the-day-off-make-up-remover/13236/',
  '/en-us/skincare/clarins/eye-contour-gel/12993/',
  '/en-us/skincare/dermalogica/age-smart-age-reversal-eye-complex/110802/',
  '/en-us/skincare/elizabeth-arden/all-gone-lip-eye-makeup-remover/37773/',
  '/en-us/skincare/gatineau/aquamemory-moisture-replenish-eye/109807/',
  '/en-us/skincare/guinot/eye-lifting/18560/',
  '/en-us/skincare/clinique/all-about-eyes/1

In [15]:
# Count items after removing duplicates

count = count_items(clean_urls)
count

{'eyes_lips': 641,
 'cleansers': 444,
 'masks': 521,
 'moisturisers': 1228,
 'sun_care': 413}

In [16]:
# Store clean list to MongoDB
  
c_category.insert_one(clean_urls)

In [17]:
# Remove the extra _id category

clean_urls.pop('_id', None)

ObjectId('5cb888ea6896a81d7065099a')

# Scrape the product details page by page

In [19]:
# Loop through all the url and store contents
# The error is fixed and I checked that all the products have been scraped despite the eroor. It took me 8 1/2 hours to scrape all links so I will not be re-running it again

sbn_url = "https://www.strawberrynet.com"
i=0
retry_urls = []
products_details = {}

for key, value in clean_urls.items():
    print(key)
    products = []
    products_details = {}
    for i in range(len(value)):
        prod, retry_url = scrape_product_details(sbn_url+value[i])
        products.append(prod)
        retry_urls.append(retry_url)
    products_details[key]= products
    c_product_details.insert_one(products_details)

product_details

eyes_lips
https://www.strawberrynet.com/en-us/skincare/elizabeth-arden/eight-hour-lipcare-stick/13488/
https://www.strawberrynet.com/en-us/skincare/shiseido/benefiance-wrinkleresist24-intensive/140188/
https://www.strawberrynet.com/en-us/skincare/clinique/all-about-eye-serum-de-puffing/104889/
https://www.strawberrynet.com/en-us/skincare/l-occitane/shea-butter-lip-balm-stick/42067/
https://www.strawberrynet.com/en-us/skincare/clarins/hydra-essentiel-moisture-replenishing/94452/
https://www.strawberrynet.com/en-us/skincare/clarins/extra-firming-eye-lift-perfecting/167004/
https://www.strawberrynet.com/en-us/skincare/estee-lauder/advanced-night-repair-eye-synchronized/172053/
https://www.strawberrynet.com/en-us/skincare/lancome/bi-facil/13977/
https://www.strawberrynet.com/en-us/skincare/clinique/take-the-day-off-make-up-remover/13236/
https://www.strawberrynet.com/en-us/skincare/clarins/eye-contour-gel/12993/
https://www.strawberrynet.com/en-us/skincare/dermalogica/age-smart-age-reversa

https://www.strawberrynet.com/en-us/skincare/l-occitane/immortelle-harvest-precious-eye/135540/
https://www.strawberrynet.com/en-us/skincare/elemis/pro-collagen-advanced-eye-treatment/150757/
https://www.strawberrynet.com/en-us/skincare/elemis/pro-collagen-eye-renewal/77347/
https://www.strawberrynet.com/en-us/skincare/nuxe/reve-de-miel-lip-moisturizing-stick/140709/
https://www.strawberrynet.com/en-us/skincare/clinique/superbalm-lip-treatment/32119/
https://www.strawberrynet.com/en-us/skincare/orlane/absolute-skin-recovery-care-eye/110097/
https://www.strawberrynet.com/en-us/skincare/prevage/anti-aging---intensive-repair-eye/166987/
https://www.strawberrynet.com/en-us/skincare/elizabeth-arden/eight-hour-cream-intensive-lip/154284/
https://www.strawberrynet.com/en-us/skincare/veld-s/eye-magic-smooth-gel---anti-aging/213221/
https://www.strawberrynet.com/en-us/skincare/peter-thomas-roth/mega-rich-intensive-anti-aging/44381/
https://www.strawberrynet.com/en-us/skincare/peter-thomas-roth/

https://www.strawberrynet.com/en-us/skincare/origins/plantscription-anti-aging-eye-treatment/148330/
https://www.strawberrynet.com/en-us/skincare/valmont/prime-lip-repair/195089/
https://www.strawberrynet.com/en-us/skincare/aesop/protective-lip-balm-spf30/199689/
https://www.strawberrynet.com/en-us/skincare/kanebo/sensai-cellular-performance-eye/71276/
https://www.strawberrynet.com/en-us/skincare/clarins/skin-smoothing-eye-mask/12996/
https://www.strawberrynet.com/en-us/skincare/fresh/sugar-lip-treatment-spf-15---petal/172387/
https://www.strawberrynet.com/en-us/skincare/fresh/sugar-lip-treatment-spf-15---tulip/186764/
https://www.strawberrynet.com/en-us/skincare/by-terry/baume-de-rose-lip-care/211335/
https://www.strawberrynet.com/en-us/skincare/lancaster/cleansing-block-eye-makeup-remover/42758/
https://www.strawberrynet.com/en-us/skincare/thalgo/collagen-eye-concentrate--salon/182012/
https://www.strawberrynet.com/en-us/skincare/thalgo/collagen-eye-gel-mask--salon-product-/182014/
h

https://www.strawberrynet.com/en-us/skincare/exuviance/age-reverse-eye-contour/161350/
https://www.strawberrynet.com/en-us/skincare/prevage/anti-aging-eye-cream-spf15-pa--/156829/
https://www.strawberrynet.com/en-us/skincare/biotherm/biocils-yeux-sensibles-eye-make-up/203798/
https://www.strawberrynet.com/en-us/skincare/kiehl-s/butterstick-lip-treatment-spf25/218913/
https://www.strawberrynet.com/en-us/skincare/christian-dior/capture-totale-360-light-up-open-up/203179/
https://www.strawberrynet.com/en-us/skincare/guinot/eye-fresh-cream/174717/
https://www.strawberrynet.com/en-us/skincare/guinot/eye-make-up-remover/151602/
https://www.strawberrynet.com/en-us/skincare/helena-rubinstein/force-c-eye-mask---daily-care/205276/
https://www.strawberrynet.com/en-us/skincare/payot/hydra-24--moisturing-reviving-eyes/202663/
https://www.strawberrynet.com/en-us/skincare/darphin/hydraskin-all-day-eye-refresh-gel-cream/213368/
https://www.strawberrynet.com/en-us/skincare/borghese/hydro-minerali-delux

https://www.strawberrynet.com/en-us/skincare/derma-e/even-tone-dark-circle-reducing/218416/
https://www.strawberrynet.com/en-us/skincare/pevonia-botanica/evolutive-eye-cream-mask--salon/111738/
https://www.strawberrynet.com/en-us/skincare/thalgo/exception-ultime-ultimate-time/171684/
https://www.strawberrynet.com/en-us/skincare/ella-bache/eye-contour-smoothing-mask--salon/220234/
https://www.strawberrynet.com/en-us/skincare/derma-e/firming-dmae-eye-lift---for-all/218428/
https://www.strawberrynet.com/en-us/skincare/laura-mercier/flawless-skin-repair-eye-creme/122162/
https://www.strawberrynet.com/en-us/skincare/lancome/genifique-advanced-youth-activating/224751/
https://www.strawberrynet.com/en-us/skincare/lancome/genifique-yeux-advanced-light-pearl/223553/
https://www.strawberrynet.com/en-us/skincare/chantecaille/gold-energizing-eye-recovery-mask/175366/
https://www.strawberrynet.com/en-us/skincare/ella-bache/green-lift-spirulina-lifting-eye/217186/
https://www.strawberrynet.com/en-us

...

https://www.strawberrynet.com/en-us/skincare/shiseido/sun-protection-lotion-n-spf-15/210434/
https://www.strawberrynet.com/en-us/skincare/clinique/targeted-protection-stick-spf-35/123139/
https://www.strawberrynet.com/en-us/skincare/biotherm/after-sun-oligo-thermal-face-cream/66543/
https://www.strawberrynet.com/en-us/skincare/sk-ii/atmosphere-cc-cream-spf50-pa---/226146/
https://www.strawberrynet.com/en-us/skincare/la-prairie/cellular-swiss-uv-protection-veil/211735/
https://www.strawberrynet.com/en-us/skincare/biotherm/creme-solaire-spf-30-uva-uvb-melting/168363/
https://www.strawberrynet.com/en-us/skincare/st--tropez/gradual-tan-plus-luminous-veil/211861/
https://www.strawberrynet.com/en-us/skincare/lancome/soleil-bronzer-smoothing---refreshing/214683/
https://www.strawberrynet.com/en-us/skincare/lancaster/sun-control-anti-wrinkles---dark/196549/
https://www.strawberrynet.com/en-us/skincare/lierac/sunissime-global-anti-aging-energizing/217938/
https://www.strawberrynet.com/en-us/ski

https://www.strawberrynet.com/en-us/skincare/thalgo/mceutic-sunscreen-spf-50--uva-uvb/204860/
https://www.strawberrynet.com/en-us/skincare/avene/moisturizing-self-tanning-silky/208449/
https://www.strawberrynet.com/en-us/skincare/fake-bake/platinum-face-anti-aging-self-tan/129908/
https://www.strawberrynet.com/en-us/skincare/helena-rubinstein/premium-uv-anti-ageing-uv-protection/163275/
https://www.strawberrynet.com/en-us/skincare/kanebo/sensai-silky-bronze-cellular-protective/169012/
https://www.strawberrynet.com/en-us/skincare/kanebo/sensai-silky-bronze-sun-protective/138384/
https://www.strawberrynet.com/en-us/skincare/lancaster/silky-touch-cream-radiant-tan-spf/153587/
https://www.strawberrynet.com/en-us/skincare/darphin/soleil-plaisir-sun-protective-cream/184510/
https://www.strawberrynet.com/en-us/skincare/fresh/sugar-sport-treatment-sunscreen/211047/
https://www.strawberrynet.com/en-us/skincare/clinique/summer-in-clinique-coffret--face/211976/
https://www.strawberrynet.com/en-us

https://www.strawberrynet.com/en-us/skincare/lancaster/sun-beauty-satin-sheen-oil-fast/137550/
https://www.strawberrynet.com/en-us/skincare/lancaster/sun-beauty-velvet-fluid-milk-spf50/212181/
https://www.strawberrynet.com/en-us/skincare/lancaster/sun-beauty-velvet-tanning-milk/166385/
https://www.strawberrynet.com/en-us/skincare/clarins/sun-care-milk-for-children-very/140919/
https://www.strawberrynet.com/en-us/skincare/lancaster/sun-care-oil-free-milky-spray-spf15/86769/
https://www.strawberrynet.com/en-us/skincare/clarins/sun-care-spray-gentle-milk-lotion/56580/
https://www.strawberrynet.com/en-us/skincare/guinot/sun-logic-age-sun-anti-ageing-sun/226185/
https://www.strawberrynet.com/en-us/skincare/guinot/sun-logic-auto-bronze-self-tanner/226191/
https://www.strawberrynet.com/en-us/skincare/phytomer/sun-radiance-self-tanning-cream/229320/
https://www.strawberrynet.com/en-us/skincare/lavera/sun-sensitiv-sun-spray-spf20--exp-/235707/
https://www.strawberrynet.com/en-us/skincare/phytom

https://www.strawberrynet.com/en-us/skincare/shiseido/urban-environment-uv-protection/234790/
https://www.strawberrynet.com/en-us/skincare/lancome/uv-expert-youth-shield-aqua-gel/234441/
https://www.strawberrynet.com/en-us/skincare/cle-de-peau/uv-protection-cream-spf-50-pa---/142547/
https://www.strawberrynet.com/en-us/skincare/cle-de-peau/uv-protective-cream-tinted-spf/231442/
https://www.strawberrynet.com/en-us/skincare/cle-de-peau/uv-protective-cream-tinted-spf/231443/
https://www.strawberrynet.com/en-us/skincare/cle-de-peau/uv-protective-cream-tinted-spf/231441/
https://www.strawberrynet.com/en-us/skincare/mustela/very-high-protection-sun-lotion/217515/
https://www.strawberrynet.com/en-us/skincare/laneige/watery-sun-cream-spf-50--pa---/229236/
https://www.strawberrynet.com/en-us/skincare/olay/white-radiance-ultra-uv-protective/166773/
https://www.strawberrynet.com/en-us/skincare/tsaio/whitening-sunscreen-make-up-lotion/225007/
https://www.strawberrynet.com/en-us/skincare/epicuren/x

NameError: name 'product_details' is not defined

In [20]:
# Check if there are any urls that haven't been scraped

retry_url

[]

# Retrieve items in MongoDB collection

In [362]:
# Get all items from product_details collection in MongoDB

all_categories = []

products = db.product_details.find({}, {'_id': 0})

for product in products:
    all_categories.append(product)
    
all_categories

[{'eyes_lips': [{'product_url': 'https://www.strawberrynet.com/en-us/skincare/elizabeth-arden/eight-hour-lipcare-stick/13488/',
    'brand_name': 'Elizabeth Arden',
    'product_name': 'Eight Hour Lipcare Stick ',
    'product_price': '9.50',
    'review_num': 169,
    'avg_stars': 4.437869822485207,
    'review_details': [[{'review_date': '11/4/2019',
       'review_header': 'lip moisture',
       'review_comment': 'excellent for the lips. give good feeling.',
       'stars_num': 5},
      {'review_date': '7/4/2019',
       'review_header': 'excellent',
       'review_comment': 'Very smooth and keeps your lips hydrated',
       'stars_num': 5},
      {'review_date': '2/4/2019',
       'review_header': 'lip stick',
       'review_comment': 'very good make the lips smooth.',
       'stars_num': 5},
      {'review_date': '17/03/2019',
       'review_header': 'love it',
       'review_comment': 'Love the way it feels and lasts long!',
       'stars_num': 5},
      {'review_date': '9/3/201

In [363]:
# Get all the values for each field and store them in DataFrames

category_names = []
brand_names = []
product_urls = []
product_names = [] 
product_prices = [] 
review_nums = [] 
avg_stars = [] 
review_details = []

for category in all_categories:
    d = category.keys()
    value = category.values()
    
    for product in list(value)[0]:
        category_names.append(list(d)[0])
        brand_names.append(product['brand_name'])
        product_urls.append(product['product_url'])
        product_names.append(product['product_name'])
        product_prices.append(product['product_price'])
        review_nums.append(int(product['review_num']))
        avg_stars.append(format(float(product['avg_stars']),'.2f'))
        if int(product['review_num']) !=0:
            review_details.append(product['review_details'])
 
# Store categories in DataFrames
category_df = pd.DataFrame({'category_name':category_names})
print(len(category_df))
category_df = category_df.drop_duplicates().reset_index(drop=True)
print(len(category_df))
category_df

3247
5


,category_name
0,eyes_lips
1,cleansers
2,masks
3,moisturisers
4,sun_care


In [364]:
# Store unique brand names in DataFrame

brand_df = pd.DataFrame({'brand_name':brand_names, 'ds_id':1})

print(len(brand_df))
brand_df = brand_df.drop_duplicates().reset_index(drop=True)
print(len(brand_df))
brand_df.head()

3247
170


,brand_name,ds_id
0,Elizabeth Arden,1
1,Shiseido,1
2,Clinique,1
3,L'Occitane,1
4,Clarins,1


In [365]:
# Store products in DataFrame

product_df = pd.DataFrame({'category_name': category_names,
                         'product_url':product_urls,
                         'brand_name':brand_names,
                         'product_name':product_names,
                         'product_price':product_prices,
                         'review_num':review_nums,
                         'avg_stars':avg_stars,
                         'ds_id':1
                        })

print(len(product_df))
product_df = product_df.drop_duplicates().reset_index(drop=True)
print(len(product_df))
product_df.head()

3247
3247


,category_name,product_url,brand_name,product_name,product_price,review_num,avg_stars,ds_id
0,eyes_lips,https://www.strawberrynet.com/en-us/skincare/e...,Elizabeth Arden,Eight Hour Lipcare Stick,9.50,169,4.44,1
1,eyes_lips,https://www.strawberrynet.com/en-us/skincare/s...,Shiseido,Benefiance WrinkleResist24 Intensive Eye Conto...,61.50,0,0.00,1
2,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,Clinique,All About Eye Serum De-Puffing Eye Massage,34.50,0,0.00,1
3,eyes_lips,https://www.strawberrynet.com/en-us/skincare/l...,L'Occitane,Shea Butter Lip Balm Stick,11.00,0,0.00,1
4,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,Clarins,Hydra-Essentiel Moisture Replenishing Lip Balm,21.50,0,0.00,1


In [366]:
#Get all the products that has reviews

r_df= product_df[product_df.review_num>0].reset_index()

r_df.head()

,index,category_name,product_url,brand_name,product_name,product_price,review_num,avg_stars,ds_id
0,0,eyes_lips,https://www.strawberrynet.com/en-us/skincare/e...,Elizabeth Arden,Eight Hour Lipcare Stick,9.50,169,4.44,1
1,9,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,Clarins,Eye Contour Gel,32.50,41,4.68,1
2,15,eyes_lips,https://www.strawberrynet.com/en-us/skincare/d...,Darphin,Wrinkle Corrective Eye Contour Cream,55.00,8,3.75,1
3,17,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,Clarins,Instant Eye Make Up Remover,28.00,41,5.05,1
4,20,eyes_lips,https://www.strawberrynet.com/en-us/skincare/d...,Decleor,Prolagene Lift Lift & Firm Eye Care,42.00,1,5.00,1


In [367]:
# Retrieve reviews

review_date = []
review_header = []
review_comment = []
stars_num = []
r_product_id = []
i=0

for p in review_details:  
    index =  r_df['index'][i] + 1
    for q in p:
        for r in q:
            review_date.append(r['review_date'])
            review_header.append(r['review_header'])
            review_comment.append(r['review_comment'])
            stars_num.append(int(r['stars_num']))
            r_product_id.append(index)
    i+=1

In [368]:
# Store reviews in DataFrame

review_df = pd.DataFrame({ 'review_date': review_date,
                          'review_header': review_header,
                          'review_comment': review_comment,
                          'stars_num':stars_num,
                          'product_id': r_product_id  
                        })
review_df.head()

,review_date,review_header,review_comment,stars_num,product_id
0,11/4/2019,lip moisture,excellent for the lips. give good feeling.,5,1
1,7/4/2019,excellent,Very smooth and keeps your lips hydrated,5,1
2,2/4/2019,lip stick,very good make the lips smooth.,5,1
3,17/03/2019,love it,Love the way it feels and lasts long!,5,1
4,9/3/2019,good product,"Is a lip that I like very much, It helps to ke...",5,1


In [369]:
# Drop duplicates, invalid dates and convert to date time
print(len(review_df))
review_df = review_df[review_df['review_date'].map(len)<=10]
review_df['review_date'] =  pd.to_datetime(review_df['review_date'], format='%d/%m/%Y')
review_df = review_df.drop_duplicates()
print(len(review_df))
review_df.head()

3996
2620


,review_date,review_header,review_comment,stars_num,product_id
0,2019-04-11,lip moisture,excellent for the lips. give good feeling.,5,1
1,2019-04-07,excellent,Very smooth and keeps your lips hydrated,5,1
2,2019-04-02,lip stick,very good make the lips smooth.,5,1
3,2019-03-17,love it,Love the way it feels and lasts long!,5,1
4,2019-03-09,good product,"Is a lip that I like very much, It helps to ke...",5,1


In [370]:
# Replace brand names with brand id

for i in range(0, len(brand_name)):
    name = brand_name['brand_name'][i]
    index =  brand_df[brand_df['brand_name']==name].index.item()+1
    product_df['brand_name'] = product_df['brand_name'].replace(name, index)
    
product_df = product_df.rename(columns={'brand_name':'brand_id'})
    
product_df.head()

,category_name,product_url,brand_id,product_name,product_price,review_num,avg_stars,ds_id
0,eyes_lips,https://www.strawberrynet.com/en-us/skincare/e...,1,Eight Hour Lipcare Stick,9.50,169,4.44,1
1,eyes_lips,https://www.strawberrynet.com/en-us/skincare/s...,2,Benefiance WrinkleResist24 Intensive Eye Conto...,61.50,0,0.00,1
2,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,3,All About Eye Serum De-Puffing Eye Massage,34.50,0,0.00,1
3,eyes_lips,https://www.strawberrynet.com/en-us/skincare/l...,4,Shea Butter Lip Balm Stick,11.00,0,0.00,1
4,eyes_lips,https://www.strawberrynet.com/en-us/skincare/c...,5,Hydra-Essentiel Moisture Replenishing Lip Balm,21.50,0,0.00,1


In [371]:
# Replace category names with category id

for i in range(0, len(category_name)):
    name = category_name['category_name'][i]
    index =  category_df[category_df['category_name']==name].index.item()+1
    product_df['category_name'] = product_df['category_name'].replace(name, index)
    
product_df = product_df.rename(columns={'category_name':'category_id'})
    
product_df.head()

,category_id,product_url,brand_id,product_name,product_price,review_num,avg_stars,ds_id
0,1,https://www.strawberrynet.com/en-us/skincare/e...,1,Eight Hour Lipcare Stick,9.50,169,4.44,1
1,1,https://www.strawberrynet.com/en-us/skincare/s...,2,Benefiance WrinkleResist24 Intensive Eye Conto...,61.50,0,0.00,1
2,1,https://www.strawberrynet.com/en-us/skincare/c...,3,All About Eye Serum De-Puffing Eye Massage,34.50,0,0.00,1
3,1,https://www.strawberrynet.com/en-us/skincare/l...,4,Shea Butter Lip Balm Stick,11.00,0,0.00,1
4,1,https://www.strawberrynet.com/en-us/skincare/c...,5,Hydra-Essentiel Moisture Replenishing Lip Balm,21.50,0,0.00,1


In [372]:
# clean product price

product_df['product_price'] = product_df['product_price'].str.replace(',','')
product_df['product_price'] = product_df['product_price'].astype(float)
product_df = product_df.rename(columns={'product_price':'price'})
product_df.head()

,category_id,product_url,brand_id,product_name,price,review_num,avg_stars,ds_id
0,1,https://www.strawberrynet.com/en-us/skincare/e...,1,Eight Hour Lipcare Stick,9.5,169,4.44,1
1,1,https://www.strawberrynet.com/en-us/skincare/s...,2,Benefiance WrinkleResist24 Intensive Eye Conto...,61.5,0,0.00,1
2,1,https://www.strawberrynet.com/en-us/skincare/c...,3,All About Eye Serum De-Puffing Eye Massage,34.5,0,0.00,1
3,1,https://www.strawberrynet.com/en-us/skincare/l...,4,Shea Butter Lip Balm Stick,11.0,0,0.00,1
4,1,https://www.strawberrynet.com/en-us/skincare/c...,5,Hydra-Essentiel Moisture Replenishing Lip Balm,21.5,0,0.00,1


# Import data into MySQL

In [373]:
import pymysql
from sqlalchemy import create_engine

user = 'root'
passw = 'password'
host =  'localhost'
port = 3306

database = 'skincare_db'

mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

category_df.to_sql(name='category', con=mydb, if_exists = 'append', index=False)
brand_df.to_sql(name='brand', con=mydb, if_exists = 'append', index=False)
product_df.to_sql(name='product', con=mydb, if_exists = 'append', index=False)
review_df.to_sql(name='review', con=mydb, if_exists = 'append', index=False)